In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [41]:
csv_file = '/home/com_2/workspace/yjshin/capstone/timeseries_dataset/20230928_cam5.csv'
pig_data = pd.read_csv(csv_file)
pig_data

,Date_time,id_0,id_1,id_2,id_3
0,2023-09-28 00:01:00,0,0,0,0
1,2023-09-28 00:02:00,0,0,0,0
2,2023-09-28 00:03:00,0,0,0,0
3,2023-09-28 00:04:00,0,0,0,0
4,2023-09-28 00:05:00,0,0,0,0
...,...,...,...,...,...
9250,2023-10-04 10:11:00,0,0,1,0
9251,2023-10-04 10:12:00,0,1,1,0
9252,2023-10-04 10:13:00,0,1,1,0
9253,2023-10-04 10:14:00,0,0,0,0


In [42]:
name = csv_file.split('/')[-1][:-4]
date = name.split('_')[0]
cam = name.split('_')[1]
print(date, cam)

if cam == 'cam1':
    num = [20, 19, 18, 17]
elif cam == 'cam2':
    num = [16, 15, 14, 13]
elif cam == 'cam3':
    num = [12, 11, 10, 9]
elif cam == 'cam4':
    num = [8, 7, 6, 5]
else:
    num = [4, 3, 2, 1]

print(num)

pig_data['Date_time'] = pd.to_datetime(pig_data['Date_time'])  # timestamp 열을 datetime 형식으로 변환
pig_data.sort_values(by='Date_time', inplace=True)  # timestamp 열을 기준으로 정렬
pig_data.reset_index(drop=True, inplace=True) 

if not pig_data['Date_time'].is_unique:
    pig_data = pig_data.loc[~pig_data['Date_time'].duplicated(keep='last')]
print(pig_data['Date_time'].is_unique)

date_time = pig_data['Date_time']
id_0 = pig_data['id_0']
id_1 = pig_data['id_1']
id_2 = pig_data['id_2']
id_3 = pig_data['id_3']
id_list = [id_0, id_1, id_2, id_3]
id_list_str = ['id_0', 'id_1', 'id_2', 'id_3']

id_date_a = [
    '2023-12-02 14:39:00', # id_0 4 8 12 16 20
    '2023-12-02 07:43:00', # id_1 3 7 11 15 19
    '2023-10-02 14:42:00', # id_2 2 6 10 14 18
    '2023-10-02 07:46:00'] # id_3 1 5 9  13 17
id_date_b = [] # 하루 전

from datetime import datetime, timedelta
for a in id_date_a:
    d = datetime.strptime(a, '%Y-%m-%d %H:%M:%S') + timedelta(days=-1)
    id_date_b.append(d.strftime('%Y-%m-%d %H:%M:%S'))
print(id_date_b)

20230928 cam5
[4, 3, 2, 1]
True
['2023-12-01 14:39:00', '2023-12-01 07:43:00', '2023-10-01 14:42:00', '2023-10-01 07:46:00']


In [43]:
for i in range(4):
    id = id_list[i]
    id_col = id_list_str[i]

    id = pd.concat([date_time, id], axis = 1)
    id.rename(columns={f'{id_col}':'ear_state'}, inplace=True)
    id['label']=0
    id.set_index('Date_time', inplace=True)
    id.loc[f'{id_date_b[i]}':f'{id_date_a[i]}','label'] = 1
    id[f'{id_date_b[i]}':f'{id_date_a[i]}']
    id.to_csv(f"/home/com_2/workspace/pig_dataset/{date}_{num[i]}.csv", mode='w')